# Importar bibliotecas

In [1]:
import time
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import os

In [2]:
from datetime import datetime

In [3]:
import zipfile, urllib.request, shutil

# Parametros

In [4]:
path_notebook = os.getcwd()

In [13]:
local_datalake = './datalake/'
local_stage = './stage/'

In [6]:
#Inicio da execução do notebook
start_general = datetime.now()

## Definir funções

In [48]:
def importar_arquivo_zip(url_name, local_file_name, local_out_file):
    print('url_name:', url_name)
    print('local_file_name:', local_file_name)
    print('local_out_file:', local_out_file)
    
    with urllib.request.urlopen(url_name) as response, open(local_file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
        
        with zipfile.ZipFile(local_file_name) as zf:
            print('')
            print('out_file:', out_file)
            zf.extractall(local_out_file)

In [8]:
def renomear_colunas(nome_df, nomes_colunas):
    for i in range(0, len(nomes_colunas), 1):
        #print (nomes_colunas[i])
        nome_df.rename(columns = {
            nome_df.columns[i] : nomes_colunas[i]
        }, inplace = True) 

# Importar base de dados de países

### Download da base de PIB por Município do IBGE de 2010 à 2018 com valores corrigidos para 2018

In [9]:
url_download = r'https://ftp.ibge.gov.br/Pib_Municipios/2018/base/base_de_dados_2010_2018_xls.zip'
nome_arquivo_local = 'pib_2010_2018.zip'

In [49]:
importar_arquivo_zip(url_download, local_datalake + nome_arquivo_local, local_datalake)
data_extracao = datetime.now()

url_name: https://ftp.ibge.gov.br/Pib_Municipios/2018/base/base_de_dados_2010_2018_xls.zip
local_file_name: ./datalake/pib_2010_2018.zip
local_out_file: ./datalake/

out_file: <_io.BufferedWriter name='./datalake/pib_2010_2018.zip'>


AttributeError: 'ZipFile' object has no attribute 'name'

### Identificar nome arquivo compactado

In [60]:
archive = zipfile.ZipFile(local_datalake + nome_arquivo_local, 'r')

In [61]:
nome_arquivo_original = archive.filelist.__getitem__(0).filename

In [62]:
pib = pd.read_excel(local_datalake + nome_arquivo_original)

In [64]:
try:
    os.remove(local_datalake + nome_arquivo_original)
except Exception as e:
    print(e)
    print('Erro deleção arquivo', local_datalake + nome_arquivo_original)
except:
    print('Erro deleção arquivo', local_datalake + nome_arquivo_original)

## Conversões

### Tratamento nomes das colunas

In [65]:
nomes_colunas = [
    'ano_PIB'
    ,'id_Regiao'
    ,'nome_Regiao'
    ,'id_Estado'
    ,'sigla_Estado'
    ,'nome_Estado'
    ,'id_Municipio'
    ,'nome_Municipio'
    ,'nome_Regiao_Metropolitana'
    ,'id_Mesoregiao'
    ,'nome_Mesoregiao'
    ,'id_Microregiao'
    ,'nome_Microregiao'
    ,'id_Regiao_Imediata'
    ,'nome_Regiao_Imediata'
    ,'municipio_Regiao_Imediata'
    ,'id_Regiao_Intermediaria'
    ,'nome_Regiao_Intermediaria'
    ,'municipio_Regiao_Intermediaria'
    ,'codigo_Concentracao_Urbana'
    ,'nome_Concentracao_Urbana'
    ,'tipo_Concentracao_Urbana'
    ,'codigo_Arranjo_Populacional'
    ,'nome_Arranjo_Populacional'
    ,'hierarquia_Urbana'
    ,'hierarquia_Urbana_Principais'
    ,'codigo_Regiao_Rural'
    ,'nome_Regiao_Rural'
    ,'nome_Regiao_Rural_Nucleo'
    ,'amazonia_Legal'
    ,'semiarido'
    ,'regiao_Sao_Paulo'
    ,'valor_Bruto_Agropecuaria'
    ,'valor_Bruto_Industria'
    ,'valor_Bruto_Servicos'
    ,'valor_Bruto_Administracao_Publica'
    ,'valor_Bruto_Total'
    ,'valor_Impostos_Liquido'
    ,'valor_PIB'
    ,'valor_PIB_percapita'
    ,'atividade_Primeiro_Maior_Valor'
    ,'atividade_Segundo_Maior_Valor'
    ,'atividade_Terceiro_Maior_Valor'
]

In [66]:
print('Quantidade de colunas do arquivo {0}, quantidade de colunas para renomear {1}'.format(len(pib.columns), len(nomes_colunas)))

Quantidade de colunas do arquivo 43, quantidade de colunas para renomear 43


In [67]:
renomear_colunas(pib, nomes_colunas)

### Ajustar colunas valore por 1000

### Na base do IBGE os valores estão divididos por 1.000 reais

In [68]:
pib['valor_Bruto_Agropecuaria'] = pib['valor_Bruto_Agropecuaria'] *1000

In [69]:
pib['valor_Bruto_Industria'] = pib['valor_Bruto_Industria'] *1000

In [70]:
pib['valor_Bruto_Servicos'] = pib['valor_Bruto_Servicos'] *1000

In [71]:
pib['valor_Bruto_Administracao_Publica'] = pib['valor_Bruto_Administracao_Publica'] *1000

In [72]:
pib['valor_Bruto_Total'] = pib['valor_Bruto_Total'] *1000

In [73]:
pib['valor_Impostos_Liquido'] = pib['valor_Impostos_Liquido'] *1000

In [74]:
pib['valor_PIB'] = pib['valor_PIB'] *1000

In [75]:
pib['populacao_Projetada'] = pib['valor_PIB'] / pib['valor_PIB_percapita'] 

In [76]:
pib['populacao_Projetada'] = pib['populacao_Projetada'].round(0)

In [77]:
pib['tipo_Valor'] = 'Preço corrente do ano referência'

In [78]:
pib['data_Extracao'] = data_extracao

In [79]:
pib['ano_Referencia'] = 2018

#### Características do dataframe

In [80]:
pib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50115 entries, 0 to 50114
Data columns (total 47 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   ano_PIB                            50115 non-null  int64         
 1   id_Regiao                          50115 non-null  int64         
 2   nome_Regiao                        50115 non-null  object        
 3   id_Estado                          50115 non-null  int64         
 4   sigla_Estado                       50115 non-null  object        
 5   nome_Estado                        50115 non-null  object        
 6   id_Municipio                       50115 non-null  int64         
 7   nome_Municipio                     50115 non-null  object        
 8   nome_Regiao_Metropolitana          12489 non-null  object        
 9   id_Mesoregiao                      50115 non-null  int64         
 10  nome_Mesoregiao                   

In [83]:
pib.isnull().sum()

ano_PIB                                  0
id_Regiao                                0
nome_Regiao                              0
id_Estado                                0
sigla_Estado                             0
nome_Estado                              0
id_Municipio                             0
nome_Municipio                           0
nome_Regiao_Metropolitana            37626
id_Mesoregiao                            0
nome_Mesoregiao                          0
id_Microregiao                           0
nome_Microregiao                         0
id_Regiao_Imediata                       0
nome_Regiao_Imediata                     0
municipio_Regiao_Imediata                0
id_Regiao_Intermediaria                  0
nome_Regiao_Intermediaria                0
municipio_Regiao_Intermediaria           0
codigo_Concentracao_Urbana           44184
nome_Concentracao_Urbana             44184
tipo_Concentracao_Urbana             44184
codigo_Arranjo_Populacional          41520
nome_Arranj

#### Especificar tratamento para valores nulos para nome da região metropolitana

In [84]:
pib['nome_Regiao_Metropolitana'].fillna('Não pertence Região Metropolitana', inplace=True)

In [85]:
pib['valida_Regiao_Metropolitana'] = np.where(
    pib['nome_Regiao_Metropolitana'] == 'Não pertence Região Metropolitana'
    ,True
    ,False
)

#### Especificar tratamento para valores nulos para código e nome da concentração urbana

In [86]:
min(pib[pib['codigo_Concentracao_Urbana'].notnull()]['codigo_Concentracao_Urbana'])

1100122.0

In [87]:
pib['codigo_Concentracao_Urbana'].fillna(0, inplace=True)

In [88]:
pib['nome_Concentracao_Urbana'].fillna('Não pertence Concentração Urbana', inplace=True)

In [89]:
pib['tipo_Concentracao_Urbana'].fillna('Não pertence Concentração Urbana', inplace=True)

In [90]:
pib['valida_Concentracao_Urbana'] = np.where(
    pib['codigo_Concentracao_Urbana'] == 0
    ,True
    ,False
)

#### Especificar tratamento para valores nulos para código e nome do arranjo populacional

In [91]:
min(pib[pib['codigo_Arranjo_Populacional'].notnull()]['codigo_Arranjo_Populacional'])

1100106.0

In [92]:
pib['codigo_Arranjo_Populacional'].fillna(0, inplace=True)

In [93]:
pib['nome_Arranjo_Populacional'].fillna('Não pertence Arranjo Populacional', inplace=True)

In [94]:
pib['valida_Arranjo_Populacional'] = np.where(
    pib['codigo_Arranjo_Populacional'] == 0
    ,True
    ,False
)

### Selecionar colunas especificas ordenar sequencia colunas

In [95]:
nomes_colunas = [
    'id_Regiao'
    ,'nome_Regiao'
    ,'id_Estado'
    ,'sigla_Estado'
    ,'nome_Estado'
    ,'id_Municipio'
    ,'nome_Municipio'
    ,'nome_Regiao_Metropolitana'
    ,'valida_Regiao_Metropolitana'
    ,'id_Mesoregiao'
    ,'nome_Mesoregiao'
    ,'id_Microregiao'
    ,'nome_Microregiao'
    ,'id_Regiao_Imediata'
    ,'nome_Regiao_Imediata'
    ,'municipio_Regiao_Imediata'
    ,'id_Regiao_Intermediaria'
    ,'nome_Regiao_Intermediaria'
    ,'municipio_Regiao_Intermediaria'
    ,'codigo_Concentracao_Urbana'
    ,'nome_Concentracao_Urbana'
    ,'tipo_Concentracao_Urbana'
    ,'valida_Concentracao_Urbana'
    ,'codigo_Arranjo_Populacional'
    ,'nome_Arranjo_Populacional'
    ,'valida_Arranjo_Populacional'
    ,'hierarquia_Urbana'
    ,'hierarquia_Urbana_Principais'
    ,'codigo_Regiao_Rural'
    ,'nome_Regiao_Rural'
    ,'nome_Regiao_Rural_Nucleo'
    ,'ano_Referencia'
    ,'ano_PIB'
    ,'valor_Bruto_Agropecuaria'
    ,'valor_Bruto_Industria'
    ,'valor_Bruto_Servicos'
    ,'valor_Bruto_Administracao_Publica'
    ,'valor_Bruto_Total'
    ,'valor_Impostos_Liquido'
    ,'valor_PIB'
    ,'valor_PIB_percapita'
    ,'populacao_Projetada'
    ,'tipo_Valor'
    ,'atividade_Primeiro_Maior_Valor'
    ,'atividade_Segundo_Maior_Valor'
    ,'atividade_Terceiro_Maior_Valor'
    ,'data_Extracao'    
]

In [96]:
pib = pib[nomes_colunas]

#### Características do dataframe após tratamento

In [97]:
pib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50115 entries, 0 to 50114
Data columns (total 47 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   id_Regiao                          50115 non-null  int64         
 1   nome_Regiao                        50115 non-null  object        
 2   id_Estado                          50115 non-null  int64         
 3   sigla_Estado                       50115 non-null  object        
 4   nome_Estado                        50115 non-null  object        
 5   id_Municipio                       50115 non-null  int64         
 6   nome_Municipio                     50115 non-null  object        
 7   nome_Regiao_Metropolitana          50115 non-null  object        
 8   valida_Regiao_Metropolitana        50115 non-null  bool          
 9   id_Mesoregiao                      50115 non-null  int64         
 10  nome_Mesoregiao                   

In [98]:
pib.isnull().sum()

id_Regiao                            0
nome_Regiao                          0
id_Estado                            0
sigla_Estado                         0
nome_Estado                          0
id_Municipio                         0
nome_Municipio                       0
nome_Regiao_Metropolitana            0
valida_Regiao_Metropolitana          0
id_Mesoregiao                        0
nome_Mesoregiao                      0
id_Microregiao                       0
nome_Microregiao                     0
id_Regiao_Imediata                   0
nome_Regiao_Imediata                 0
municipio_Regiao_Imediata            0
id_Regiao_Intermediaria              0
nome_Regiao_Intermediaria            0
municipio_Regiao_Intermediaria       0
codigo_Concentracao_Urbana           0
nome_Concentracao_Urbana             0
tipo_Concentracao_Urbana             0
valida_Concentracao_Urbana           0
codigo_Arranjo_Populacional          0
nome_Arranjo_Populacional            0
valida_Arranjo_Populacion

## Salvar dataframe em arquivo local formato parquet

In [99]:
pib.to_parquet(local_stage + 'pib-municipios-2010-2018.parquet', index=False)

In [100]:
#Fim da execução do notebook
end_general = datetime.now()

In [101]:
# Write the DataFrame to a BigQuery table

print("Tempo total notebook " + str(end_general - start_general))

Tempo total notebook 0:32:16.433327
